The code in this folder, `tmQM_rev`, is code corresponding to the paper 'Applying Large Graph Neural Networks to Predict Transition Metal Complex Energies Using the tmQM_rev Dataset,' by Aaron G. Garrison, Javier Heras-Domingo, John R. Kitchin, Gabriel dos Passos Gomes, Zachary W. Ulissi, and Samuel M. Blau.

The code included in this work is broken into several subdirectories, for ease of navigation. The purpose of this document is to inform the user of which files are stored in which directories, as well as how to interpret and use them. This document only gives a surface-level overview of how to use the files, for more detail, see the documents in the `notebooks` folder.

# all_data

This folder is where datasets consisting of all of the data, whether that be for tmQM, tmQM_rev, or the removed structures, is held. There are several subfolders:

## removed_structures

This folder contains a single file, `removed_structures.db`. This is an ASE database that contains the 158 structures removed from tmQM when tmQM_rev was created. An example of accessing this file for a specific structure is shown below.

In [1]:
from ase.db import connect
from ase.io import read

db = connect('/home/jovyan/tmQM_rev/all_data/removed_structures/removed_structures.db')
entry = db.get(id=1)
structure = read('/home/jovyan/tmQM_rev/all_data/removed_structures/removed_structures.db@id=1')

print(structure)
print(entry.data)

[Atoms(symbols='C44H20N2O4Ti', pbc=False)]
{'CSD_code': 'MQUPTI', 'q': 0, 'S': 0, 'Stoichiometry': 'C44H20N2O4Ti', 'MND': 6, 'Electronic_E': -2946.606812, 'Dispersion_E': -0.191406, 'Dipole_M': 3.8986, 'Metal_q': 0.92542, 'HL_Gap': 0.02629, 'HOMO_Energy': -0.18177, 'LUMO_Energy': -0.15548, 'Polarizability': 486.904053, 'Recomputed_E': -2945.5453118794}


Each entry in this database has the ASE Atoms object presented in tmQM for that structure (after translation such that the metal center was at the origin), as well as all of the data presented in tmQM for the structure (CSD code, charge, spin, chemical formula, metal node degree, electronic energy, dispersion energy, dipole moment, metal charge, HOMO-LUMO gap, HOMO energy, LUMO energy, and polarizability), along with the electronic energy computed at the $\omega$B97M-V / def2-SVPD level of theory, if applicable. For the three structures where the energy did not converge, the recomputed energy is listed as 'NOT CONVERGED'.

## tmQM

This folder contains several files, all of which contain the structures in tmQM. There are ASE database files, `tmQM.db` and `neutral_tmQM.db`, which contain all 86,665 structures in tmQM, along with the properties included with each structure in the original tmQM paper. These database files are accessed in exactly the same way as shown above.

The original structures from tmQM, without translation, are included in `tmQM_combined_notcentered.extxyz`. This file was made by taking the original `.xyz` files from tmQM and concatenating the two. One could also reference the original tmQM files, which can be found at: https://github.com/bbskjelstad/tmqm. The structures after translating the metal center to be at the origin can be found in `tmQM_centered.extxyz`. Note that both of the `.extxyz` files do not contain the DFT properties of the structures, just the original structures. These `.extxyz` files can be read with `ase.io.read`.

The dataset is also included as an LMDB. There are two LMDB files (each with an associated `.lmdb-lock` file). `tmqm_elec_e_reference_corrected.lmdb` contains all of tmQM and the properties included with that dataset, and uses the reference corrected electronic energies as the targets. `neutral_tmqm_elec_e_reference_corrected.lmdb` contains the same information, but only includes neutral structures..

One can read the LMDB files with the following method (note this requires the installation of OCP from https://github.com/Open-Catalyst-Project/ocp):

In [2]:
import sys
sys.path.append('/home/jovyan/ocp')
from ocpmodels.datasets.lmdb_dataset import LmdbDataset

tmqm = LmdbDataset({"src":"/home/jovyan/tmQM_rev/all_data/tmQM/tmqm_elec_e_reference_corrected.lmdb"})

tmqm[0]

Data(pos=[116, 3], cell=[1, 3, 3], atomic_numbers=[116], natoms=116, sid=0, CSD_code='WIXKOE', Electronic_E=-2045.524942, Dispersion_E=-0.239239, Dipole_M=4.2333, Metal_q=2.10934, HL_Gap=0.13108, HOMO_Energy=-0.16204, LUMO_Energy=-0.03096, Polarizability=598.457913, y_relaxed=-0.008107466590217882)

We see that in the LMDB objects, the structures are stored as graphs, in a list of atomic positions and atomic numbers. The properties from tmQM are included as attributes of the graph, and the target is included as `y_relaxed`.

## tmQM_rev

This folder contains several files, all of which contain the structures in tmQM_rev, along with only the properties calculated in this work, not those calculated for tmQM. Again, there are ASE database files, `tmQM_rev.db` and `neutral_tmQM_rev.db`, which contain all of tmQM_rev and only the neutral structures, respectively.

A `.traj` file is also included containing all of the structures in tmQM_rev along with their properties, `tmQM_rev.traj`. This file can be read using `ase.io.Trajectory`.

Two LMDBs, with the same naming conventions as for tmQM, are also included.

In [3]:
from ase.io import Trajectory
tmqm_rev = Trajectory('/home/jovyan/tmQM_rev/all_data/tmQM_rev/tmQM_rev.traj')

print(tmqm_rev[0])
print(tmqm_rev[0].info)

Atoms(symbols='C22H36AgCl4N4', pbc=False)
{'q': 1, 'spin': 1, 'CSD_code': 'TECJUJ', 'energy': -3064.7028208554, 'formula': 'AgH36C22(NCl)4', 'SID': 0, 'target': 0.1996804086247721}


# chemiscope_files

This folder contains two files, `tmqm_gemnet_evpredictions.json.gz` and `tmqm_rev_gemnet_evpredictions.json.gz`. Both of these files can be downloaded and viewed with the chemiscope viewer, at https://chemiscope.org/. They have several properties, but the recommended viewing is setting the x and y axes as the tSNE maps, and then looking at heatmaps or 3D plots of how other properties change across that mapping of chemical space. These files contain the residuals of a GemNet-T model, trained on 80% of the respective dataset (either all of tmQM or all of tmQM_rev), used to predict the energies of all of the structures in that dataset.

# configs

This folder contains all of the configuration files used to train the GNNs used in this work (SchNet, PaiNN, SpinConv, and GemNet-T) in OCP. Each config will tell the percent of the dataset it used for training data (e.g. `20-` denotes a configuration file using 20% of the data for that dataset to train), as well as the model it is for (e.g. `-schnet` is used for a SchNet model). There are four main subfolders, `tmQM`, containing configs for models trained on tmQM, `tmQM_neutral`, containing configs for models trained on the neutral structures in tmQM, `tmQM_rev`, containing configs for models trained on tmQM_rev, and `tmQM_rev_neutral`, containing congigs for models trained on the neutral structures in tmQM_rev. Each of these folders contains 16 files, marked with the percentage of training data that file will use and the model it corresponds to.

# guides
This folder contains several Jupyter notebooks and markdown files describing the methodology used in this work as well as more detailed instructions on how to use the code in the `scripts` folder.

# oc20-pretrained

This folder contains the fine-tuning results from this work. This contains GemNet-T models trained on 80% of the entirety of tmQM_rev and the neutral subset of tmQM_rev, denoted by the prefix `80-gemnet-oc20preds` and `neutral-80-gemnet-oc20preds`, respectively. For each of the two models, there are three files: a `.yml`, which is the config for the model trained, a `.pt` containing the trained checkpoint, and a `.npz` containing the test set predictions from the best-performing epoch.

# predictions

This folder contains predictions, for each model trained, on its respective test set, using the checkpoints in the `trained_checkpoints` folder. Each subfolder denotes the dataset used to train that model, in the same format as in the `configs` folder, and then each filename has a `train%-model` structure.

These predictions are included as `.npz` files, which can be accessed as follows:

In [4]:
import numpy as np
data = np.load('/home/jovyan/tmQM_rev/predictions/tmQM_rev/80-gemnet.npz')

ids = data['ids']
preds = data['energy']

print(ids[:5])
print(preds[:5])

['10018' '10026' '10036' '10042' '10051']
[-0.05812447  0.00445005 -0.05136636  0.13523446  0.04485682]


This gives the SID identifiers, which can be found as attributes of the graphs in the LMDBs, along with the predicted (reference corrected) energy. To match the prediction with the original target, one finds the structures in the test set, matches the SID, and then compares the actual and predicted target. If only the MAE is desired, or one wants to generate a parity plot, this functionality is included in the `parity_plots.py` file in the `scripts` folder.

# reference_correction

This folder contains the data calculated from and used in the reference correction strategy. There are four files, corresponding to the data needed for either all of or the neutral subset of tmQM and tmQM_rev. The files are given as `.npz` files, and can be read as follows:

In [5]:
data = np.load('/home/jovyan/tmQM_rev/reference_correction/tmqm_rev_elec_e_corrections.npz')

elems = data['elem_list']
num_elems = data['coeff_matrix']
corrections = data['target_references']

print(elems)
print(corrections)

print(num_elems[:5])

[ 1  5  6  7  8  9 14 15 16 17 21 22 23 24 25 26 27 28 29 30 33 34 35 39
 40 41 42 43 44 45 46 47 48 53 57 72 73 74 75 76 77 78 79 80]
[-5.97321536e-01 -2.48476683e+01 -3.80705845e+01 -5.47001193e+01
 -7.51609788e+01 -9.97473456e+01 -2.89435902e+02 -3.41234462e+02
 -3.98051942e+02 -4.60073981e+02 -7.60618091e+02 -8.49368761e+02
 -9.43847541e+02 -1.04423138e+03 -1.15072667e+03 -1.26344096e+03
 -1.38244680e+03 -1.50802660e+03 -1.64013238e+03 -1.77906852e+03
 -2.23549319e+03 -2.40116289e+03 -2.57375097e+03 -3.83083893e+01
 -4.70347736e+01 -5.69322050e+01 -6.81145437e+01 -8.06866490e+01
 -9.47652814e+01 -1.10409719e+02 -1.27755252e+02 -1.46749667e+02
 -1.67611815e+02 -2.97741100e+02 -3.14935490e+01 -4.80188180e+01
 -5.69499962e+01 -6.70059106e+01 -7.82186595e+01 -9.05329428e+01
 -1.04244639e+02 -1.19219756e+02 -1.35562958e+02 -1.53301067e+02]
[[36.  0. 22.  4.  0.  0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
 

There are three arrays stored in each of the files. The first, `elem_list`, gives the atomic numbers of the elements present in the dataset. The second, `coeff_matrix`, gives a (number of structures) x (number of elements present) matrix that gives the number of each element that appears in each structure. The third, `target_references`, gives the energies subtracted off for each atom of an element that appeared in order to get the reference corrected electronic energy.

# scripts

This folder contains a number of python files used in this work. Their purpose and use is explained more in-depth in the `notebooks` folder. A quick summary is:
- `center_geometries.py` was used to translate the structures in tmQM such that their metal centers were at the origin.
- `parity_plots.py` was used to generate parity plots for trained models.
- `random_split_subsets.py` was used to generate train/val/test splits from the data.
- `reference_correction.py` was used for the reference correction strategy.
- `revert_reference_correction.py` can be used to convert the predicted targets back to absolute energies.
- `tmqm_asap_residual_append.py` and `tmqm_rev_asap_residual_append.py` are used to append model predictions to ASE Atoms objects, which can then be used with the ASAP package (at https://github.com/BingqingCheng/ASAP) that would deliver the residuals of the model in the resulting chemiscope file, for tmQM and tmQM_rev respectively.
- `tmqm_lmdb_creation.py` and `tmqm_rev_lmdb_creation.py` are both used to take the ASE representations for tmQM or tmQM_rev, respectively, and turn them into LMDBs.

# trained_checkpoints

This folder contains checkpoints for all of the models trained, named in the same convention as the configuration files. A guide of how to use these files in model training can be found in `"guides/Training Models on tmQM_rev.md"`. The folder structure is the same as the other folders.

# training_data

This folder contains the data (and data splits) used to train all models. There are four subfolders, corresponding to either tmQM or tmQM_rev, and either the complete dataset or the neutral subset for both. Each of the subfolders has files inside it with the format `train%-split`, in that `80-train` is the training split for the 80/10/10 split, and would be used with the `80-val` and `80-test` splits.